# Preparation

## Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.math as tfm
import datetime

from pandas import DataFrame
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SimpleRNN, GRU, Bidirectional, Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer

2022-06-11 00:40:24.777871: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Data preparation

In [2]:
from Feature import Feature


# Dataframe with 15 minutes between each row
df = pd.read_csv("../data/raw_binance_btc_1h.csv")
df

,Time_UTC_Start,open,high,low,close,volumn,quote_asset_volumn,number_of_trades,base_asset_volumn,quote_asset_volumn.1,label
0,2018-07-18 11:00:00,7403.62,7409.77,7400.74,7405.01,35.159664,2.603868e+05,229.0,16.229202,120194.647778,1.0
1,2018-07-18 12:00:00,7418.02,7419.99,7416.55,7419.98,44.474849,3.299686e+05,114.0,37.061210,274965.375198,1.0
2,2018-07-18 13:00:00,7435.90,7437.02,7431.00,7437.02,42.365364,3.149200e+05,276.0,26.980340,200556.171114,1.0
3,2018-07-18 14:00:00,7458.14,7459.36,7445.03,7451.30,25.105532,1.871272e+05,228.0,9.664364,72045.500859,0.0
4,2018-07-18 15:00:00,7451.71,7453.01,7442.10,7444.30,60.280231,4.488050e+05,220.0,31.972354,238020.383175,0.0
...,...,...,...,...,...,...,...,...,...,...,...
33328,2022-05-07 03:00:00,35953.63,35953.64,35943.60,35951.24,6.376430,2.292318e+05,339.0,3.818980,137290.253366,0.0
33329,2022-05-07 04:00:00,35812.22,35826.14,35812.21,35826.14,6.066650,2.173069e+05,348.0,5.129450,183735.644012,1.0
33330,2022-05-07 05:00:00,35746.62,35768.06,35722.24,35768.05,34.156320,1.220607e+06,993.0,19.901990,711168.120653,1.0
33331,2022-05-07 06:00:00,35898.96,35903.80,35890.03,35900.00,16.707700,5.997658e+05,580.0,10.902570,391364.049595,1.0


In [3]:
a = Feature(df.copy())
for i in [4]:
    a.price_EMA(days=i, inplace=True)
    a.price_SMA(days=i, inplace=True)
    a.price_WMA(days=i, inplace=True)
    a.volumn_EMA(days=i, inplace=True)
    a.volumn_SMA(days=i, inplace=True)
    a.volumn_WMA(days=i, inplace=True)
    a.RS(days=i, inplace=True)
    a.RSI(days=i, inplace=True)
    a.std(days=i, inplace=True)
    a.raw_money_flow(inplace=True)
    a.plus_DM(inplace=True)
    a.minus_DM(inplace=True)
    a.price_diff(inplace=True)
    a.smoothed_plus_DM(days=i, inplace=True)
    a.smoothed_minus_DM(days=i, inplace=True)
    a.TR(inplace=True)
    a.std(days=i, inplace=True)
    a.mad(days=i, inplace=True)
    a.ATR(days=i, inplace=True)
    a.CCI(days=i, inplace=True)
    a.DX(days=i, inplace=True)
    a.ADX(days=i, inplace=True)
    a.PPO(inplace=True)
    a.BOP(days=i, inplace=True)
    a.MFI(days=i, inplace=True)
    a.MACD()
# a.dropna(inplace=True)

In [12]:
a[["","CCI4"]]

0               NaN
1               NaN
2               NaN
3        102.457293
4         50.411862
            ...    
33328   -118.601974
33329   -133.333333
33330    -89.773294
33331     41.504062
33332    -22.544891
Name: CCI4, Length: 33333, dtype: float64

In [5]:
a[["std4","mad4","close"]].iloc[:10,:]

,std4,mad4,close
0,NaN,NaN,7405.01
1,NaN,NaN,7419.98
2,NaN,NaN,7437.02
3,17.440484,15.724167,7451.30
4,11.642281,11.128333,7444.30
5,8.857487,9.350000,7427.40
6,8.690493,8.471667,7441.46
7,25.468388,22.682500,7494.65
8,27.293498,24.813333,7429.65
9,40.762455,30.560000,7379.99


In [6]:
m = a["close"].to_numpy()[0:3].mean()

In [7]:
abs(a["close"].to_numpy()[0:3] - m).sum()/3

10.900000000000546

In [8]:
DataFrame.mad(df["close"][0:4])

15.83250000000021

In [9]:
a.to_csv("data_with_features_ver1.csv")

In [10]:
len(a.columns)

38

In [11]:
import pandas as pd
pd.read_csv("data_with_features_ver1.csv")

,Unnamed: 0,Time_UTC_Start,open,high,low,close,volumn,quote_asset_volumn,number_of_trades,base_asset_volumn,...,mad4,ATR4,CCI4,plus_DI4,minus_DI4,DX4,ADX4,PPO12-26,BOP4,MFI4
0,0,2018-07-18 11:00:00,7403.62,7409.77,7400.74,7405.01,35.159664,2.603868e+05,229.0,16.229202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2018-07-18 12:00:00,7418.02,7419.99,7416.55,7419.98,44.474849,3.299686e+05,114.0,37.061210,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2018-07-18 13:00:00,7435.90,7437.02,7431.00,7437.02,42.365364,3.149200e+05,276.0,26.980340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2018-07-18 14:00:00,7458.14,7459.36,7445.03,7451.30,25.105532,1.871272e+05,228.0,9.664364,...,15.724167,8.2050,102.457293,725.563681,0.000000,100.000000,NaN,NaN,0.108106,NaN
4,4,2018-07-18 15:00:00,7451.71,7453.01,7442.10,7444.30,60.280231,4.488050e+05,220.0,31.972354,...,11.128333,8.6750,50.411862,428.731988,59.106628,75.767958,NaN,NaN,-0.100175,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33328,33328,2022-05-07 03:00:00,35953.63,35953.64,35943.60,35951.24,6.376430,2.292318e+05,339.0,3.818980,...,25.814167,14.0450,-118.601974,259.683161,1027.643289,59.655430,50.399383,-0.715491,0.208717,100.0
33329,33329,2022-05-07 04:00:00,35812.22,35826.14,35812.21,35826.14,6.066650,2.173069e+05,348.0,5.129450,...,62.000417,10.7850,-133.333333,191.029207,2789.197960,87.180225,69.773168,-0.706009,0.324963,100.0
33330,33330,2022-05-07 05:00:00,35746.62,35768.06,35722.24,35768.05,34.156320,1.220607e+06,993.0,19.901990,...,99.125000,19.0375,-89.773294,108.220617,1634.024951,87.576881,66.536629,-0.703486,0.556447,100.0
33331,33331,2022-05-07 06:00:00,35898.96,35903.80,35890.03,35900.00,16.707700,5.997658e+05,580.0,10.902570,...,68.289167,20.8900,41.504062,1137.123025,1058.437051,3.583868,87.279389,-0.664306,0.326115,100.0
